In [1]:
#Uploading Local file

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving bank.csv to bank.csv
User uploaded file "bank.csv" with length 684858 bytes


In [0]:
import tensorflow as tf

In [3]:
import keras

Using TensorFlow backend.


In [0]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [0]:
bank= pd.read_csv('bank.csv')

In [6]:
bank.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
X= bank.drop(['RowNumber','Exited','CustomerId','Surname','HasCrCard'], axis=1)
y= bank['Exited']

In [8]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,1,112542.58
2,502,France,Female,42,8,159660.80,3,0,113931.57
3,699,France,Female,39,1,0.00,2,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,79084.10


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
encoder = LabelEncoder()
encoder.fit(X['Geography'])
X['Geography'] = encoder.transform(X['Geography'])

In [0]:
encoder = LabelEncoder()
encoder.fit(X['Gender'])
X['Gender'] = encoder.transform(X['Gender'])

In [12]:
X.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,101348.88
1,608,2,0,41,1,83807.86,1,1,112542.58
2,502,0,0,42,8,159660.80,3,0,113931.57
3,699,0,0,39,1,0.00,2,0,93826.63
4,850,2,0,43,2,125510.82,1,1,79084.10


In [13]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
X_train.shape

(8000, 9)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense

In [0]:
def create_baseline():
	# create model
  model = Sequential()
  model.add(Dense(60, input_dim=9, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, activation='tanh'))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [19]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Results: 79.98% (0.02%)


In [23]:
def create_smaller():
	# create model
  model = Sequential()
  model.add(Dense(30, input_dim=9, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, activation='tanh'))
  model.add(Dense(30, input_dim=9, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=20, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
results = cross_val_score(pipeline, X_train, y_train, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.86% (0.76%)


In [21]:
results

array([0.85571519, 0.86508433, 0.83437501, 0.85490933, 0.86053784])

In [25]:
l=create_smaller()
l.fit(X_train, y_train, epochs=10)

Epoch 1/10
8000/8000 [==============================] - 1s 176us/step - loss: 0.5735 - acc: 0.7997
Epoch 2/10
8000/8000 [==============================] - 0s 42us/step - loss: 0.5009 - acc: 0.7997
Epoch 3/10
8000/8000 [==============================] - 0s 42us/step - loss: 0.5009 - acc: 0.7997
Epoch 4/10
8000/8000 [==============================] - 0s 42us/step - loss: 0.5009 - acc: 0.7997
Epoch 5/10
8000/8000 [==============================] - 0s 42us/step - loss: 0.5008 - acc: 0.7997
Epoch 6/10
8000/8000 [==============================] - 0s 42us/step - loss: 0.5010 - acc: 0.7997
Epoch 7/10
8000/8000 [==============================] - 0s 43us/step - loss: 0.5009 - acc: 0.7997
Epoch 8/10
8000/8000 [==============================] - 0s 43us/step - loss: 0.5009 - acc: 0.7997
Epoch 9/10
8000/8000 [==============================] - 0s 43us/step - loss: 0.5009 - acc: 0.7997
Epoch 10/10
8000/8000 [==============================] - 0s 43us/step - loss: 0.5011 - acc: 0.7997


In [26]:
test_loss, test_acc = l.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

2000/2000 [==============================] - 1s 307us/step
Test accuracy: 0.7825


In [27]:
bank.corr()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.006495,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.006495,-0.014883,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


In [0]:
a=l.predict(X_test)

In [29]:
a

array([[0.20337352],
       [0.20337352],
       [0.20337352],
       ...,
       [0.20337352],
       [0.20337352],
       [0.20337352]], dtype=float32)

In [30]:
m=create_baseline()
m.fit(X_train, y_train, epochs=10)

Epoch 1/10
8000/8000 [==============================] - 2s 232us/step - loss: 0.5653 - acc: 0.7997
Epoch 2/10
8000/8000 [==============================] - 0s 43us/step - loss: 0.5009 - acc: 0.7997
Epoch 3/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5010 - acc: 0.7997
Epoch 4/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.5011 - acc: 0.7997
Epoch 5/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5010 - acc: 0.7997
Epoch 6/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.5009 - acc: 0.7997
Epoch 7/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5010 - acc: 0.7997
Epoch 8/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5008 - acc: 0.7997
Epoch 9/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5010 - acc: 0.7997
Epoch 10/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5012 - acc: 0.7997


In [0]:
a=m.predict(X_test)

In [32]:
a

array([[0.19119343],
       [0.19119343],
       [0.19119343],
       ...,
       [0.19119343],
       [0.19119343],
       [0.19119343]], dtype=float32)

In [0]:
Final_Result= a>0.5